In [ ]:
!pip install accelerate peft bitsandbytes trl py7zr auto-gptq optimum

In [ ]:
import torch
import os
from datasets import load_dataset,Dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments,GPTQConfig
from peft import LoraConfig,AutoPeftModelForCausalLM,prepare_model_for_kbit_training,get_peft_model
from trl import SFTTrainer

In [ ]:
data  = load_dataset('samsum',split='train')

In [ ]:
data


Dataset({
    features: ['id', 'dialogue', 'summary'],
    num_rows: 14732
})

In [ ]:
data.to_pandas()


,id,dialogue,summary
0,13818513,Amanda: I baked cookies. Do you want some?\r\...,Amanda baked cookies and will bring Jerry some...
1,13728867,Olivia: Who are you voting for in this electio...,Olivia and Olivier are voting for liberals in ...
2,13681000,"Tim: Hi, what's up?\r\nKim: Bad mood tbh, I wa...",Kim may try the pomodoro technique recommended...
3,13730747,"Edward: Rachel, I think I'm in ove with Bella....",Edward thinks he is in love with Bella. Rachel...
4,13728094,Sam: hey overheard rick say something\r\nSam:...,"Sam is confused, because he overheard Rick com..."
...,...,...,...
14727,13863028,Romeo: You are on my ‘People you may know’ lis...,Romeo is trying to get Greta to add him to her...
14728,13828570,Theresa: <file_photo>\r\nTheresa: <file_photo>...,Theresa is at work. She gets free food and fre...
14729,13819050,John: Every day some bad news. Japan will hunt...,Japan is going to hunt whales again. Island an...
14730,13828395,Jennifer: Dear Celia! How are you doing?\r\nJe...,Celia couldn't make it to the afternoon with t...


In [ ]:
data_df = data.to_pandas()

In [ ]:
data_df.head()

,id,dialogue,summary
0,13818513,Amanda: I baked cookies. Do you want some?\r\...,Amanda baked cookies and will bring Jerry some...
1,13728867,Olivia: Who are you voting for in this electio...,Olivia and Olivier are voting for liberals in ...
2,13681000,"Tim: Hi, what's up?\r\nKim: Bad mood tbh, I wa...",Kim may try the pomodoro technique recommended...
3,13730747,"Edward: Rachel, I think I'm in ove with Bella....",Edward thinks he is in love with Bella. Rachel...
4,13728094,Sam: hey overheard rick say something\r\nSam:...,"Sam is confused, because he overheard Rick com..."


In [ ]:
data_df[['dialogue','summary']]

,dialogue,summary
0,Amanda: I baked cookies. Do you want some?\r\...,Amanda baked cookies and will bring Jerry some...
1,Olivia: Who are you voting for in this electio...,Olivia and Olivier are voting for liberals in ...
2,"Tim: Hi, what's up?\r\nKim: Bad mood tbh, I wa...",Kim may try the pomodoro technique recommended...
3,"Edward: Rachel, I think I'm in ove with Bella....",Edward thinks he is in love with Bella. Rachel...
4,Sam: hey overheard rick say something\r\nSam:...,"Sam is confused, because he overheard Rick com..."
...,...,...
14727,Romeo: You are on my ‘People you may know’ lis...,Romeo is trying to get Greta to add him to her...
14728,Theresa: <file_photo>\r\nTheresa: <file_photo>...,Theresa is at work. She gets free food and fre...
14729,John: Every day some bad news. Japan will hunt...,Japan is going to hunt whales again. Island an...
14730,Jennifer: Dear Celia! How are you doing?\r\nJe...,Celia couldn't make it to the afternoon with t...


In [ ]:
data_df["text"] = data_df[["dialogue", "summary"]].apply(lambda x: "###Human: Summarize this following dialogue: " + x["dialogue"] + "\n###Assistant: " +x["summary"], axis=1)

In [ ]:
data_df.head()

,id,dialogue,summary,text
0,13818513,Amanda: I baked cookies. Do you want some?\r\...,Amanda baked cookies and will bring Jerry some...,###Human: Summarize this following dialogue: A...
1,13728867,Olivia: Who are you voting for in this electio...,Olivia and Olivier are voting for liberals in ...,###Human: Summarize this following dialogue: O...
2,13681000,"Tim: Hi, what's up?\r\nKim: Bad mood tbh, I wa...",Kim may try the pomodoro technique recommended...,###Human: Summarize this following dialogue: T...
3,13730747,"Edward: Rachel, I think I'm in ove with Bella....",Edward thinks he is in love with Bella. Rachel...,###Human: Summarize this following dialogue: E...
4,13728094,Sam: hey overheard rick say something\r\nSam:...,"Sam is confused, because he overheard Rick com...",###Human: Summarize this following dialogue: S...


In [ ]:
print(data_df.iloc[0][3])

###Human: Summarize this following dialogue: Amanda: I baked  cookies. Do you want some?
Jerry: Sure!
Amanda: I'll bring you tomorrow :-)
###Assistant: Amanda baked cookies and will bring Jerry some tomorrow.


<ipython-input-68-101905aafc9b>:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(data_df.iloc[0][3])


In [ ]:
data = Dataset.from_pandas(data_df)

In [ ]:
!pip install  huggingface_hub

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('mistralai/Mistral-7B-Instruct-v0.1')

In [ ]:
tokenizer.eos_token

'</s>'

In [ ]:
tokenizer.pad_token_id

2

In [ ]:
tokenizer.pad_token

'</s>'

In [ ]:
tokenizer.eos_token_id

2

In [ ]:
tokenizer.eos_token

'</s>'

In [ ]:
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [ ]:
quantization_config_loading = GPTQConfig(bits=4, disable_exllama=True, tokenizer=tokenizer)

Using `disable_exllama` is deprecated and will be removed in version 4.37. Use `use_exllama` instead and specify the version with `exllama_config`.The value of `use_exllama` will be overwritten by `disable_exllama` passed in `GPTQConfig` or stored in your config file.


In [ ]:
model = AutoModelForCausalLM.from_pretrained(
                          "TheBloke/Mistral-7B-Instruct-v0.1-GPTQ",
                          quantization_config=quantization_config_loading,
                          device_map="auto")

/usr/local/lib/python3.11/dist-packages/transformers/quantizers/auto.py:195: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.However, loading attributes (e.g. ['use_cuda_fp16', 'use_exllama', 'max_input_length', 'exllama_config', 'disable_exllama']) will be overwritten with the one you passed to `from_pretrained`. The rest will be ignored.
  warnings.warn(warning_msg)
/usr/local/lib/python3.11/dist-packages/accelerate/utils/modeling.py:1536: UserWarning: Current model requires 3630956672 bytes of buffer for offloaded layers, which seems does not fit any GPU's remaining memory. If you are experiencing a OOM later, please consider using offload_buffers=True.
  warnings.warn(
Some weights of the model checkpoint at TheBloke/Mistral-7B-Instruct-v0.1-GPTQ were not used when initializing MistralForCausalLM: {'model.layer

In [ ]:
print(model)

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32000, 4096, padding_idx=0)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralAttention(
          (k_proj): QuantLinear()
          (o_proj): QuantLinear()
          (q_proj): QuantLinear()
          (v_proj): QuantLinear()
        )
        (mlp): MistralMLP(
          (act_fn): SiLU()
          (down_proj): QuantLinear()
          (gate_proj): QuantLinear()
          (up_proj): QuantLinear()
        )
        (input_layernorm): MistralRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): MistralRMSNorm((4096,), eps=1e-05)
      )
    )
    (norm): MistralRMSNorm((4096,), eps=1e-05)
    (rotary_emb): MistralRotaryEmbedding()
  )
  (lm_head): Linear(in_features=4096, out_features=32000, bias=False)
)


In [ ]:
#used to supress the model
model = prepare_model_for_kbit_training(model)

In [ ]:
peft_config = LoraConfig(
    r=16,lora_alpha=16,lora_dropout=0.05,bias="none",task_type="CAUSAL_LM",target_modules=["q_proj","k_proj","v_proj"]
)

In [ ]:
model = get_peft_model(model,peft_config)

In [ ]:
print(model)

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): MistralForCausalLM(
      (model): MistralModel(
        (embed_tokens): Embedding(32000, 4096, padding_idx=0)
        (layers): ModuleList(
          (0-31): 32 x MistralDecoderLayer(
            (self_attn): MistralAttention(
              (k_proj): lora.QuantLinear(
                (base_layer): QuantLinear()
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=1024, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
                (quant_linear_module): QuantLinear()
             

In [ ]:
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,    # Enable 4-bit quantization
    bnb_4bit_compute_dtype=torch.float16,  # Use fp16 for computation
    bnb_4bit_use_double_quant=True,  # Use double quantization for memory efficiency
)

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    "mistralai/Mistral-7B-Instruct-v0.1",
    quantization_config=quantization_config,
    device_map="auto"  # Automatically assigns layers to available GPUs
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
model.config.use_cache=False

In [ ]:
model.config.pretraining_tp=1

In [ ]:
model.gradient_checkpointing_enable()

In [ ]:
model = prepare_model_for_kbit_training(model)

In [ ]:
peft_config = LoraConfig(
        r=16, lora_alpha=16, lora_dropout=0.05, bias="none", task_type="CAUSAL_LM", target_modules=["q_proj", "v_proj"]
    )

In [ ]:
model = get_peft_model(model, peft_config)

In [ ]:
model = prepare_model_for_kbit_training(model)

In [ ]:

training_arguments = TrainingArguments(
    output_dir="mistral-finetuned-samsum",
    per_device_train_batch_size=4,  # Reduced batch size
    gradient_accumulation_steps=2,  # Increased gradient accumulation steps
    optim="paged_adamw_32bit",
    learning_rate=2e-4,
    lr_scheduler_type="cosine",
    save_strategy="epoch",
    logging_steps=100,
    num_train_epochs=1,
    max_steps=250,
    fp16=True,
    push_to_hub=True,
    report_to="none",
    max_grad_norm=1.0,
    gradient_checkpointing=True,  # Enable gradient checkpointing
    # dataloader_num_workers=2,      # Adjust based on your system's CPU cores
)

# Initialize SFTTrainer
trainer = SFTTrainer(
    model=model,
    train_dataset=data,
    peft_config=peft_config,
    args=training_arguments,
    tokenizer=tokenizer,
)


<ipython-input-114-f98954279740>:21: FutureWarning: `tokenizer` is deprecated and removed starting from version 0.16.0 for `SFTTrainer.__init__`. Use `processing_class` instead.
  trainer = SFTTrainer(


Converting train dataset to ChatML:   0%|          | 0/14732 [00:00<?, ? examples/s]

Applying chat template to train dataset:   0%|          | 0/14732 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/14732 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/14732 [00:00<?, ? examples/s]

In [ ]:
trainer.train()

Step,Training Loss
100,1.866400
200,1.763300


TrainOutput(global_step=250, training_loss=1.8016432189941407, metrics={'train_runtime': 2697.3583, 'train_samples_per_second': 0.741, 'train_steps_per_second': 0.093, 'total_flos': 577958902824960.0, 'train_loss': 1.8016432189941407})

In [ ]:
! cp -r /content/mistral-finetuned-samsum

cp: missing destination file operand after '/content/mistral-finetuned-samsum'
Try 'cp --help' for more information.


In [ ]:
trainer.push_to_hub()

CommitInfo(commit_url='https://huggingface.co/IshtiyaqMarzuq/mistral-finetuned-samsum/commit/acbc5d621f89f736e2d04f5c62ca54edf0d4c5cf', commit_message='End of training', commit_description='', oid='acbc5d621f89f736e2d04f5c62ca54edf0d4c5cf', pr_url=None, repo_url=RepoUrl('https://huggingface.co/IshtiyaqMarzuq/mistral-finetuned-samsum', endpoint='https://huggingface.co', repo_type='model', repo_id='IshtiyaqMarzuq/mistral-finetuned-samsum'), pr_revision=None, pr_num=None)

In [ ]:
from peft import AutoPeftModelForCausalLM
from transformers import GenerationConfig
from transformers import AutoTokenizer
import torch
tokenizer = AutoTokenizer.from_pretrained("/content/mistral-finetuned-samsum")

In [ ]:
inputs = tokenizer("""
###Human: Summarize this following dialogue: Sunny: I'm at the railway station in Chennai Karthik: No problems so far? Sunny: no, everything's going smoothly Karthik: good. lets meet there soon!
###Assistant: """, return_tensors="pt").to("cuda")


In [ ]:
model = AutoPeftModelForCausalLM.from_pretrained(
    "/content/mistral-finetuned-samsum",
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map="cuda")

Some weights of the model checkpoint at TheBloke/Mistral-7B-Instruct-v0.1-GPTQ were not used when initializing MistralForCausalLM: {'model.layers.12.self_attn.v_proj.bias', 'model.layers.10.mlp.gate_proj.bias', 'model.layers.2.self_attn.v_proj.bias', 'model.layers.26.self_attn.k_proj.bias', 'model.layers.19.self_attn.q_proj.bias', 'model.layers.24.self_attn.v_proj.bias', 'model.layers.16.mlp.down_proj.bias', 'model.layers.31.self_attn.v_proj.bias', 'model.layers.9.mlp.gate_proj.bias', 'model.layers.22.mlp.gate_proj.bias', 'model.layers.5.self_attn.v_proj.bias', 'model.layers.30.mlp.up_proj.bias', 'model.layers.11.self_attn.k_proj.bias', 'model.layers.27.self_attn.v_proj.bias', 'model.layers.21.mlp.gate_proj.bias', 'model.layers.26.self_attn.q_proj.bias', 'model.layers.23.mlp.down_proj.bias', 'model.layers.3.self_attn.k_proj.bias', 'model.layers.19.self_attn.k_proj.bias', 'model.layers.2.mlp.up_proj.bias', 'model.layers.7.self_attn.q_proj.bias', 'model.layers.5.self_attn.o_proj.bias', '

In [ ]:
generation_config = GenerationConfig(
    do_sample=True,
    top_k=1,
    temperature=0.1,
    max_new_tokens=25,
    pad_token_id=tokenizer.eos_token_id
)

In [ ]:
import time
st_time = time.time()
outputs = model.generate(**inputs, generation_config=generation_config)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))
print(time.time()-st_time)


###Human: Summarize this following dialogue: Sunny: I'm at the railway station in Chennai Karthik: No problems so far? Sunny: no, everything's going smoothly Karthik: good. lets meet there soon!
###Assistant:  Sunny is at the railway station in Chennai. Everything is going smoothly. Karthik will meet him soon.
29.265328884124756


In [ ]:
inputs = tokenizer("""
###Human: Summarize this following dialogue:
Alice: The presentation is almost ready for the team meeting.
Bob: Great, do you need any help with the visuals?
Alice: No, I've got that covered. Thanks for checking, though!
Bob: Awesome, let me know if you need anything else.

###Assistant: """, return_tensors="pt").to("cuda")


In [ ]:
import time
st_time = time.time()
outputs = model.generate(**inputs, generation_config=generation_config)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))
print(time.time()-st_time)


###Human: Summarize this following dialogue: Alice: The presentation is almost ready for the team meeting. Bob: Great, do you need any help with the visuals? Alice: No, I've got that covered. Thanks for checking, though! Bob: Awesome, let me know if you need anything else.
###Assistant:  Alice is almost ready for the team meeting. Bob will help her with the visuals.  Alice doesn't need anything
28.50069499015808


In [ ]:
inputs = tokenizer("""
###Human: Summarize this following dialogue:
Ishtiyaq: Today i fine-tuned a mistral model,which can generate a summarization of a conversation between two people.
Marzuq: Great,how can i access your model?
Ishtiyaq: you can check my git repo or at huggingface where i have pushed it!
Marzuq: Awesome.
###Assistant: """, return_tensors="pt").to("cuda")


In [ ]:
import time
st_time = time.time()
outputs = model.generate(**inputs, generation_config=generation_config)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))
print(time.time()-st_time)


###Human: Summarize this following dialogue:
Ishtiyaq: Today i fine-tuned a mistral model,which can generate a summarization of a conversation between two people.
Marzuq: Great,how can i access your model? 
Ishtiyaq: you can check my git repo or at huggingface where i have pushed it! 
Marzuq: Awesome.
###Assistant: 
Ishtiyaq has fine-tuned a mistral model that can generate a summarization of a conversation
28.947664260864258


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import torch

# Define the path in Google Drive
save_path = '/content/drive/My Drive/Mistral-Model.pth'

# Save the model
torch.save(model.state_dict(), save_path)
print(f"Model saved to {save_path}")


Model saved to /content/drive/My Drive/Mistral-Model.pth
